In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

## Montgomery
1. Merge masks
2. Remove black borders
3. Resize to 256x256
4. Equalize hist and [1,99] percintile
5. Save

## Other datasets:
Same but without 1

In [2]:
def scale_uint(img):
    img = np.asarray(img,dtype=np.float) * 255 / np.max(img)
    img = img.astype(np.uint8)
    return img

In [5]:
input_img = '../OriginalData/Montgomery/Images/'
#input_mask = '../OriginalData/Shenzen/mask/'
mask_suffix = '_mask.png'
input_left = '../OriginalData/Montgomery/ManualMask/leftMask/'
input_right = '../OriginalData/Montgomery/ManualMask/rightMask/'
output_img = '../Data/Images/'
output_mask = '../Data/Masks/'
output_masked = '../Data/Masked/'

img_names = os.listdir(input_img)
dist_size=(256,256)

In [6]:
for filename in tqdm(img_names):
    img = cv2.imread(os.path.join(input_img,filename),cv2.IMREAD_GRAYSCALE)
    img = scale_uint(img)
    
    #1 merge masks
    left_mask = cv2.imread(os.path.join(input_left,filename),cv2.IMREAD_GRAYSCALE)
    left_mask = left_mask > 0
    right_mask = cv2.imread(os.path.join(input_right,filename),cv2.IMREAD_GRAYSCALE)
    right_mask = right_mask > 0
    mask = left_mask | right_mask
    #mask = cv2.imread(os.path.join(input_mask,filename[:-4] + mask_suffix),cv2.IMREAD_GRAYSCALE)
    if mask is None:
        continue
    else:
        mask = mask > 0
    
    #2 Remove black borders
    y,x = np.nonzero(img)
    y_0, y_1 = y.min(), y.max()
    x_0, x_1 = x.min(), x.max()
    img = img[y_0:y_1+1,x_0:x_1+1]
    mask = mask[y_0:y_1+1,x_0:x_1+1].astype(float)
    
    #3 Resize
    img = cv2.resize(img,dist_size,cv2.INTER_AREA)
    mask = cv2.resize(mask,dist_size,cv2.INTER_AREA) > 0.5
    
    #4 Equalize hist
    a,b = np.percentile(img,[1,99])
    img = np.clip(img,a,b)
    img = scale_uint(img)
    img = cv2.equalizeHist(img)
    
    #5 Save
    masked = np.stack([img]*3,axis=2)
    masked[mask,2] = 0
    
    cv2.imwrite(os.path.join(output_img,filename),img)
    cv2.imwrite(os.path.join(output_mask,filename),mask.astype(np.uint8) * 255)
    cv2.imwrite(os.path.join(output_masked,filename),masked)

100%|██████████| 138/138 [00:52<00:00,  2.62it/s]
